In [7]:
import pandas as pd
from PyPDF2 import PdfReader

def read_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        index = page_text.find("Horario por centro de costo: Semanal (Excel)")
        full_text += page_text[:index]
    return full_text

def format_schedule(schedule: str):
    entrada = schedule.split("-")[0]
    salida = schedule.split("-")[1]

    am_format = "AM"
    pm_format = "PM"

    hora = int(entrada.split(":")[0])
    if (int(hora) >= 12):
        hora = "12"
        entrada = f"{hora}:{entrada.split(':')[1]}"
    
    if (entrada[-1] == "a"):
        entrada = entrada[:-1] + am_format
    else:
        entrada = entrada[:-1] + pm_format

    hora = int(salida.split(":")[0])
    if (int(hora) >= 12):
        hora = "12"
        salida = f"{hora}:{salida.split(':')[1]}"
    if (salida[-1] == "a"):
        salida = salida[:-1] + am_format
    else:
        salida = salida[:-1] + pm_format

    return f"{entrada} - {salida}"

def format_cell(cellString: str):
    if cellString == "0[":
        return "DIA DE DESCANSO"
    if cellString == "0]":
        return "VACACIONES"
    if cellString == "0/":
        return "PAGO HORAS FERIADO"
    return format_schedule(cellString)

pdf_path = "./horarios/Horario 11-18.pdf"
first_cajero = "BARRIENTOS JERI, MILAGROS NICOL"
first_rs = "YANAC DAVILA, GERALD RONNY"

pdf_text = read_pdf(pdf_path)

print(pdf_text[pdf_text.find("dom")+14:])

start_index = pdf_text.find(first_cajero)
end_index = pdf_text.find(first_rs)

pdf = pdf_text[start_index:end_index]

pdf = pdf.replace("RS", " ")
pdf = pdf.replace("SELF", " ")
pdf = pdf.replace("CAJEROS", " ")
pdf = pdf.replace(" x", " ")
pdf = pdf.replace("DIA DE\nDESCANSO 0:00", " 0[ ")
pdf = pdf.replace("DIA DE \nDESCANSO 0:00", " 0[ ")
pdf = pdf.replace("VACACIONES 0:00", " 0] ")
pdf = pdf.replace("PAGO HORAS\nFERIADO", " 0/ ")
pdf = pdf.replace("PAGO HORAS \nFERIADO", " 0/ ")

pdf_list = pdf.split()

i=0
n = len(pdf_list)

while (i<n):
    if pdf_list[i] == "0/" and pdf_list[i+1][0].isdigit():
        # Eliminar tiempo de PAGO HORAS FERIADO
        id = pdf_list[i+1].find(":") + 3
        pdf_list[i+1] = pdf_list[i+1][id:]
        if pdf_list[i+1] == "":
            pdf_list.pop(i+1)
            n-=1
    if not pdf_list[i][0].isdigit():
        # Unir celdas que no son horas
        if not (i==0 or pdf_list[i-1][0].isdigit()):
            pdf_list[i-1] += f" {pdf_list[i]}"
            pdf_list.pop(i)
            n-=1
            if pdf_list[i-1][-2].isdigit():
                pdf_list.insert(i, pdf_list[i-1][-13:])
                pdf_list[i-1] = pdf_list[i-1][:-13]
                n+=1
            continue
    i+=1

# Create a DataFrame
df = pd.DataFrame([pdf_list[i:i+8] for i in range(0, n, 8)])


CARHUANCHO RAYMUNDO, JAMES 
JESÚS03:30p-07:15p    x DIA DE 
DESCANSO 0:0010:15a-02:00p    x 02:30p-06:15p    x 02:30p-06:15p    x 10:45a-02:30p    x 06:15p-10:00p    x
EVANGELISTA INFANTES, MARIA 
FERNANDA10:45a-02:30p    x 10:45a-02:30p    x 10:45a-02:30p    x 10:45a-02:30p    x 10:45a-02:30p    x 06:15p-10:00p    x DIA DE 
DESCANSO 0:00
HUAMANI QUICANO, EMELYN HEIDY 07:00a-10:45a    x 07:00a-10:45a    x 07:00a-10:45a    x 07:00a-10:45a    x 07:00a-10:45a    x DIA DE 
DESCANSO 0:0007:00a-10:45a    x
MACHCO HUARCA, NICOLAS AXEL DIA DE 
DESCANSO 0:0007:15p-11:00p    x 07:15p-11:00p    x 07:15p-11:00p    x 07:15p-11:00p    x 07:15p-11:00p    x 02:30p-06:15p    x
ORIHUELA PUMACCAHUA, ALVARO 
RAUL07:15p-11:00p    x 06:15p-10:00p    x 06:15p-10:00p    x DIA DE 
DESCANSO 0:0006:15p-10:00p    x 02:30p-06:15p    x 10:45a-02:30p    x
TRUJILLO PENADILLO, SHEYLA 
YAJAIRA01:30p-05:15p    x 02:30p-06:15p    x 02:30p-06:15p    x 06:15p-10:00p    x DIA DE 
DESCANSO 0:0007:00a-10:45a    x 07:15p-11:0

In [8]:
df = df.rename(columns={0: "Nombres", 1: "Lunes", 2: "Martes", 3: "Miércoles", 4: "Jueves", 5: "Viernes", 6: "Sábado", 7: "Domingo"})

In [9]:
df

,Nombres,Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo
0,"BARRIENTOS JERI, MILAGROS NICOL",12:00p-03:30p,0[,0[,0[,0[,12:30p-09:30p,12:30p-09:30p
1,"CHAVEZ ONOFRE, CAMILA GERALDINE",06:00p-10:30p,0[,0[,0[,0[,08:00a-04:00p,12:15p-09:15p
2,"ERIQUE CALLE, MARIA ANTONIETA",0[,0[,0[,0[,02:00p-11:00p,11:00a-03:15p,08:45a-05:00p
3,"HINOSTROZA MARIN, CAMILA MARIA",0[,0[,0[,0[,08:00a-12:30p,07:30a-04:15p,11:45a-08:00p
4,"HUAMANI TORRES, LUIS RODRIGO",0[,0[,0[,12:00p-03:30p,0[,11:15a-08:15p,01:00p-10:00p
5,"MENDOZA DIEGO, ZAIDA VANESSA",05:15p-09:00p,0[,0[,0[,12:00p-09:00p,0[,12:00p-08:45p
6,"MEZA MELO, NORMA FERNANDA",0[,0[,0[,0[,05:30p-10:15p,02:30p-10:15p,02:45p-11:45p
7,"QUISPE MENDOZA, ANTONY MAURICIO",0[,0[,12:00p-08:45p,0[,0[,10:15a-07:15p,11:30a-03:15p
8,"SAAVEDRA ALVAN, ANA MARIA",11:30a-08:30p,0[,0[,05:45p-09:45p,0[,03:15p-11:45p,0[
9,"ZAVALA SOSA, NICOLE",0[,0[,0[,0[,03:00p-11:45p,11:15a-08:15p,06:30p-10:15p


In [10]:
df["Lunes"] = df["Lunes"].apply(format_cell)
df["Martes"] = df["Martes"].apply(format_cell)
df["Miércoles"] = df["Miércoles"].apply(format_cell)
df["Jueves"] = df["Jueves"].apply(format_cell)
df["Viernes"] = df["Viernes"].apply(format_cell)
df["Sábado"] = df["Sábado"].apply(format_cell)
df["Domingo"] = df["Domingo"].apply(format_cell)

In [11]:
df

,Nombres,Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo
0,"BARRIENTOS JERI, MILAGROS NICOL",12:00PM - 03:30PM,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,12:30PM - 09:30PM,12:30PM - 09:30PM
1,"CHAVEZ ONOFRE, CAMILA GERALDINE",06:00PM - 10:30PM,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,08:00AM - 04:00PM,12:15PM - 09:15PM
2,"ERIQUE CALLE, MARIA ANTONIETA",DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,02:00PM - 11:00PM,11:00AM - 03:15PM,08:45AM - 05:00PM
3,"HINOSTROZA MARIN, CAMILA MARIA",DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,08:00AM - 12:30PM,07:30AM - 04:15PM,11:45AM - 08:00PM
4,"HUAMANI TORRES, LUIS RODRIGO",DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,12:00PM - 03:30PM,DIA DE DESCANSO,11:15AM - 08:15PM,01:00PM - 10:00PM
5,"MENDOZA DIEGO, ZAIDA VANESSA",05:15PM - 09:00PM,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,12:00PM - 09:00PM,DIA DE DESCANSO,12:00PM - 08:45PM
6,"MEZA MELO, NORMA FERNANDA",DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,05:30PM - 10:15PM,02:30PM - 10:15PM,02:45PM - 11:45PM
7,"QUISPE MENDOZA, ANTONY MAURICIO",DIA DE DESCANSO,DIA DE DESCANSO,12:00PM - 08:45PM,DIA DE DESCANSO,DIA DE DESCANSO,10:15AM - 07:15PM,11:30AM - 03:15PM
8,"SAAVEDRA ALVAN, ANA MARIA",11:30AM - 08:30PM,DIA DE DESCANSO,DIA DE DESCANSO,05:45PM - 09:45PM,DIA DE DESCANSO,03:15PM - 11:45PM,DIA DE DESCANSO
9,"ZAVALA SOSA, NICOLE",DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,DIA DE DESCANSO,03:00PM - 11:45PM,11:15AM - 08:15PM,06:30PM - 10:15PM


In [12]:
# export to csv with separtor ;
df.to_csv("schedule.csv", sep=";", index=False)